In [46]:
import torch
import numpy as np
import numba
import torch
import ewald_summation as es

In [ ]:
%load_ext line_profiler

In [97]:
configuration = np.random.uniform(0, 3.5, (1000, 3))
sigma = np.array([1] * len(configuration))
sigma_arr = 0.5 * (sigma[:, None] + sigma)
epsilon = np.array([1] * len(configuration))
epsilon_arr = np.sqrt(epsilon[:, None] * epsilon)
cutoff_lj = 3.5
switch_width_lj = 1

In [94]:
def lj_potential_pairwise(distance, sigma_lj, epsilon_lj):
    if(distance <= 0 or distance > cutoff_lj):
        return 0.
    else:
        inv_dist = sigma_lj / distance
        inv_dist2 = inv_dist * inv_dist
        inv_dist4 = inv_dist2 * inv_dist2
        inv_dist6 = inv_dist2 * inv_dist4
        phi_LJ = 4. * epsilon_lj * inv_dist6 * (inv_dist6 - 1.)
        if(distance <= cutoff_lj - switch_width_lj):
            return phi_LJ
        else:
            t = (distance - cutoff_lj) / switch_width_lj
            switch = t * t * (3. + 2. * t)
            return phi_LJ * switch
        
def lj_potential_loops(x):
    output = np.zeros(len(x))
    for i in range(len(x)):
        potential = 0
        for j in range(i, len(x)):
            sigma_lj = sigma_arr[i, j]
            epsilon_lj = epsilon_arr[i, j]
            distance = np.linalg.norm(x[i, :] - x[j, :])
            potential += lj_potential_pairwise(distance,sigma_lj, epsilon_lj)
        output[i] = potential
    return output


In [95]:
x = np.random.uniform(0, 4, (100, 3))
np.testing.assert_allclose(np.sum(lj_potential_loops(x)), es.potentials.lj.lj_potential_total(x))

In [99]:
%lprun -f lj_potential_loops lj_potential_loops(configuration)

In [100]:
timing_loops = %timeit np.sum(lj_potential_loops(configuration))

3.37 s ± 50.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [126]:
def potential_along_axis(x):
        # potenital w/o switch
        if x[0] > 0 and x[0] <= cutoff_lj - switch_width_lj:
            x[0] = 4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6)
        # potential with switch
        elif x[0] > cutoff_lj - switch_width_lj and x[0] <= cutoff_lj:
            t = (x[0] - cutoff_lj * x[1]) / (cutoff_lj * x[1] - cutoff_lj - switch_width_lj * x[1])
            switch = 2 * t ** 3 + 3 * t ** 2
            x[0] = switch * (4 * x[2] * x[1]**6 * (x[1]**6 / x[0]**12 - 1 / x[0]**6))
        # potential after cutoff
        elif x[0] > cutoff_lj * x[1]:
            x[0] = 0
        return x[0]

def lj_potential_np_along_axis(x, sigma_arr, epsilon_arr):
    # initialize output as array with distances and corresponding sigma, epsilon along axis=2
    output = np.zeros((x.shape[0], x.shape[0], 3))
    output[:, :, 0] = np.linalg.norm(x[:, None, :] - x[None, :, :], axis=-1)
    output[:, :, 1] = sigma_arr
    output[:, :, 2] = epsilon_arr
    # calculate potentials
    output[:, :, 0] = np.apply_along_axis(potential_along_axis, 2, output)
    output = np.sum(output[:, :, 0], axis=-1)
    return output

In [127]:
x = np.random.uniform(0, 4, (100, 3))
sigma_temp = np.array([1] * len(x))
sigma_arr_temp= 0.5 * (sigma_temp[:, None] + sigma_temp)
epsilon_temp = np.array([1] * len(x))
epsilon_arr_temp = np.sqrt(epsilon_temp[:, None] * epsilon_temp)

np.testing.assert_allclose(0.5 * np.sum(lj_potential_np_along_axis(x, sigma_arr_temp, epsilon_arr_temp)), 
                           es.potentials.lj.lj_potential_total(x))

In [129]:
%lprun -f lj_potential_np_along_axis lj_potential_np_along_axis(configuration, sigma_arr, epsilon_arr)

In [130]:
timings_np_apply_along_axis = %timeit lj_potential_np_along_axis(configuration, sigma_arr, epsilon_arr)

6.29 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
